<a href="https://colab.research.google.com/github/Rajeshwari-Mohanraj14/Youtube_summarizer/blob/main/Youtube_Video_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube_transcript_api


In [ ]:
!pip install transformers

In [ ]:
# Import necessary libraries
import re
import nltk
import numpy as np
from youtube_transcript_api import YouTubeTranscriptApi
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
from transformers import BartTokenizer, BartForConditionalGeneration, pipeline

# Download NLTK resources
nltk.download('punkt')

# Fetch transcript from YouTube
link = "https://youtu.be/SIyxjRJ8VNY?si=Mssg5DOfhqNnYpWz"
unique_id = link.split("=")[-1]
sub = YouTubeTranscriptApi.get_transcript(unique_id)
subtitle = " ".join([x['text'] for x in sub])

# Process the transcript
subtitle = subtitle.replace("\n", "")
sentences = sent_tokenize(subtitle)

# Create a dictionary to map sentences to their indices
organized_sent = {k: v for v, k in enumerate(sentences)}

# Configure TF-IDF vectorizer
tf_idf = TfidfVectorizer(min_df=1,
                         strip_accents='unicode',
                         max_features=None,
                         lowercase=True,
                         token_pattern=r'\w{1,}',
                         ngram_range=(1, 3),
                         use_idf=True,
                         smooth_idf=True,
                         sublinear_tf=True,
                         stop_words='english')

# Compute sentence vectors and scores
sentence_vectors = tf_idf.fit_transform(sentences)
sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

# Select top N sentences
N = 3
top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]

# Map scored sentences with their indices and order them
mapped_sentences = [(sentence, organized_sent[sentence]) for sentence in top_n_sentences]
mapped_sentences = sorted(mapped_sentences, key=lambda x: x[1])
ordered_sentences = [element[0] for element in mapped_sentences]

# Generate a summary using top N sentences
tfidf_summary = " ".join(ordered_sentences)

# BART Summarization
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Encode the subtitle and generate the summary
input_tensor = tokenizer.encode(subtitle, return_tensors="pt", max_length=512, truncation=True)
outputs_tensor = model.generate(input_tensor, max_length=160, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode the BART summary
bart_summary = tokenizer.decode(outputs_tensor[0], skip_special_tokens=True)

# Alternative BART Summarization using pipeline
summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)

# Define a function to split text into manageable chunks
def split_text(text, max_chunk_size):
    words = text.split()
    chunks = []
    chunk = []
    chunk_size = 0

    for word in words:
        word_size = len(word) + 1  # Plus one for the space
        if chunk_size + word_size > max_chunk_size:
            chunks.append(' '.join(chunk))
            chunk = []
            chunk_size = 0
        chunk.append(word)
        chunk_size += word_size

    if chunk:
        chunks.append(' '.join(chunk))

    return chunks

# Split the subtitle into chunks that the model can handle
max_chunk_size = 512  # Adjust based on the model's max length
chunks = split_text(subtitle, max_chunk_size)

# Summarize each chunk and concatenate the results
summaries = []
for chunk in chunks:
    summary = summarizer(chunk, max_length=180, min_length=30, do_sample=False)
    summaries.append(summary[0]['summary_text'])

# Join all summaries into a single summary
pipeline_summary = ' '.join(summaries)

# Print the summaries
print("TF-IDF Summary:")
print(tfidf_summary)
print("\nBART Summary:")
print(bart_summary)
print("\nPipeline BART Summary:")
print(pipeline_summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


TranscriptsDisabled: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=Mssg5DOfhqNnYpWz! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!